In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

column_labels = ['Lucky Star 1', 'Lucky Star 2', 'Ball 1', 'Ball 2', 'Ball 3', 'Ball 4', 'Ball 5']
column_indices = {name: i for i, name in enumerate(column_labels)}

2023-10-20 20:50:41.252803: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-20 20:50:41.252849: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-20 20:50:41.252864: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-20 20:50:41.256618: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Num GPUs Available:  1


2023-10-20 20:50:42.298965: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-20 20:50:42.301505: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-20 20:50:42.301533: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
df = pd.read_csv('euromillions-dataset.csv')
df

,Lucky Star 1,Lucky Star 2,Ball 1,Ball 2,Ball 3,Ball 4,Ball 5
0,5,7,2,6,14,19,23
1,5,8,3,23,24,34,35
2,2,5,10,15,31,41,42
3,8,11,12,14,21,45,48
4,2,11,5,14,36,40,42
...,...,...,...,...,...,...,...
164,6,9,1,19,36,38,49
165,9,10,10,25,29,34,45
166,2,4,6,16,18,39,47
167,7,12,5,14,35,36,39


In [3]:
n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

num_features = df.shape[1]

In [4]:
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

In [5]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [82]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [83]:
w2 = WindowGenerator(input_width=7, label_width=1, shift=1,
                     label_columns=column_labels)
w2

Total window size: 8
Input indices: [0 1 2 3 4 5 6]
Label indices: [7]
Label column name(s): ['Lucky Star 1', 'Lucky Star 2', 'Ball 1', 'Ball 2', 'Ball 3', 'Ball 4', 'Ball 5']

In [84]:
# Stack three slices, the length of the total window.
example_window = tf.stack([np.array(train_df[:w2.total_window_size]),
                           np.array(train_df[20:20+w2.total_window_size]),
                           np.array(train_df[40:40+w2.total_window_size])])

example_inputs, example_labels = w2.split_window(example_window)

print('All shapes are: (batch, time, features)')
print(f'Window shape: {example_window.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'Labels shape: {example_labels.shape}')

All shapes are: (batch, time, features)
Window shape: (3, 8, 7)
Inputs shape: (3, 7, 7)
Labels shape: (3, 1, 7)


In [85]:
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.utils.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

In [86]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [87]:
# Each element is an (inputs, label) pair.
w2.train.element_spec

(TensorSpec(shape=(None, 7, 7), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 1, 7), dtype=tf.float32, name=None))

In [88]:
lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(128, return_sequences=False),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1)
])

In [89]:
wide_window = WindowGenerator(
    input_width=4, label_width=1, shift=1,
    label_columns=column_labels)

wide_window

print('Input shape:', wide_window.example[0].shape)
print('Output shape:', lstm_model(wide_window.example[0]).shape)
print(wide_window.example[0])

Input shape: (32, 4, 7)
Output shape: (32, 1)
tf.Tensor(
[[[ 0.8983111   0.46139607 -1.3519758  -1.4262052  -0.10944976
   -0.68553746 -0.9942326 ]
  [-1.1869488  -0.6573314   1.1071093   0.6137004  -0.34426925
   -0.942342    0.21524623]
  [ 1.940941    0.8343052  -1.3519758  -1.8086874  -1.7531861
   -0.17192844  0.5176159 ]
  [ 0.20322447 -1.0302405   0.18495236 -0.02377009  0.24277946
   -0.3003307  -1.1454175 ]]

 [[ 1.940941    0.8343052  -1.0445901  -1.4262052  -2.1054153
   -1.3275487   1.1223553 ]
  [ 0.20322447  0.46139607  0.3386452   1.5061591   0.82982814
    0.34168056  0.6688008 ]
  [-1.1869488  -2.148968   -0.4298189   0.7411945   0.9472379
    0.9836918   0.8199856 ]
  [-0.8394054  -0.6573314   1.7218806   0.6137004   0.9472379
    0.8552896   0.9711705 ]]

 [[ 0.5507678   0.46139607 -1.0445901  -1.5536993  -1.9880055
   -0.3003307   0.06406138]
  [-0.49186212 -1.4031497  -0.89089733 -1.4262052  -1.8705958
   -2.7399735  -1.2966022 ]
  [-0.49186212 -1.0302405  -0.27612

In [90]:
MAX_EPOCHS = 120

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [91]:
history = compile_and_fit(lstm_model, wide_window)

Epoch 1/120
4/4 [==============================] - 1s 86ms/step - loss: 0.9878 - mean_absolute_error: 0.8278 - val_loss: 0.8006 - val_mean_absolute_error: 0.7540
Epoch 2/120
4/4 [==============================] - 0s 28ms/step - loss: 0.9884 - mean_absolute_error: 0.8310 - val_loss: 0.8006 - val_mean_absolute_error: 0.7540
Epoch 3/120
4/4 [==============================] - 0s 32ms/step - loss: 0.9888 - mean_absolute_error: 0.8253 - val_loss: 0.8005 - val_mean_absolute_error: 0.7541
Epoch 4/120
4/4 [==============================] - 0s 29ms/step - loss: 0.9930 - mean_absolute_error: 0.8290 - val_loss: 0.8005 - val_mean_absolute_error: 0.7541
Epoch 5/120
4/4 [==============================] - 0s 29ms/step - loss: 0.9887 - mean_absolute_error: 0.8323 - val_loss: 0.8004 - val_mean_absolute_error: 0.7541
Epoch 6/120
4/4 [==============================] - 0s 22ms/step - loss: 0.9884 - mean_absolute_error: 0.8270 - val_loss: 0.8005 - val_mean_absolute_error: 0.7541
Epoch 7/120
4/4 [===========

In [92]:
sample = tf.constant([[[i for i in j] for j in ((df[:4] - train_mean) / train_std).to_numpy()]])
print(sample)

tf.Tensor(
[[[ 0.20322447 -0.6573314  -1.19828297 -1.42620515 -1.40095688
   -1.96955998 -2.95963563]
  [ 0.20322447 -0.28442224 -1.04459015  0.7411945  -0.2268595
   -0.04352619 -1.14541743]
  [-0.83940543 -1.40314972  0.03125956 -0.27875828  0.59500868
    0.85528958 -0.08712347]
  [ 1.24585437  0.83430524  0.33864519 -0.40625238 -0.57908871
    1.36889859  0.81998563]]], shape=(1, 4, 7), dtype=float64)


In [93]:
prediction = lstm_model.predict(sample)
(prediction * tf.constant([train_std])) + tf.constant([train_mean])

1/1 [==============================] - 0s 163ms/step


<tf.Tensor: shape=(1, 7), dtype=float64, numpy=
array([[ 4.60610185,  8.94057773, 10.22817107, 17.70668288, 26.49712936,
        34.85554572, 43.01499114]])>

As seen above, and since the data set is very small, it is very probable for the network to fit using average values for each ball slot. One way around this problem is to add the difference between each ball and its surrounding value to the network inputs. This way there is more weight given to the ball values with respect to each other rather than just the average value of each ball slot.

Another way to reduce this effect is to augment the data by, for example adding a constant value to each ball slot and repeating this to generate more data which carry the same trends in the data but not necessarily repeat the same ball values.

A more natural way is to use a GAN to generate more data that have resemblance or closeness to the trends we see in the data. This way the essence of the trends is captured in the network rather than using some constant value to add to each data point.